In [1]:
!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
 46  908k   46  424k    0     0   230k      0  0:00:03  0:00:01  0:00:02  230k
100  908k  100  908k    0     0   468k      0  0:00:01  0:00:01 --:--:--  468k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3  878k    3 34137    0     0  38212      0  0:00:23 --:--:--  0:00:23 38227
100  878k  100  878k    0     0   708k      0  0:00:01  0:00:01 --:--:--  708k


In [ ]:
import cv2, dlib, sys
import numpy as np   
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# Face detection XML load and trained model loading
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile=False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:/Users/82109/Desktop/face_detector-master/shape_predictor_68_face_landmarks.dat')

# Video capture using webcam
camera = cv2.VideoCapture(0)




# overlay function
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
    try :
        bg_img = background_img.copy()
        # convert 3 channels to 4 channels
        if bg_img.shape[2] == 3:
            bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

        if overlay_size is not None:
            img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

        b, g, r, a = cv2.split(img_to_overlay_t)

        mask = cv2.medianBlur(a, 5)

        h, w, _ = img_to_overlay_t.shape
        roi = bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]

        img1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
        img2_fg = cv2.bitwise_and(img_to_overlay_t, img_to_overlay_t, mask=mask)

        bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)] = cv2.add(img1_bg, img2_fg)

        # convert 4 channels to 4 channels
        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)
        return bg_img
    except Exception : return background_img

face_roi = []
face_sizes = []
result=1

while True:
    # Capture image from camera
    ret, frame = camera.read()

    # Convert color to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Face detection in frame
    faces = face_detection.detectMultiScale(gray,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(30,30))
    
    # Create empty image
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    
    # Perform emotion recognition only when face is detected
    if len(faces) > 0:
        # For the largest image
        face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        # Resize the image to 48x48 for neural network
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Emotion predict
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        

        # Assign labeling
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
 
        # Label printing
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = "{}: {:.2f}%".format(emotion, prob * 100)    
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
            
    if not ret:
        break

    # resize frame
    ori = frame.copy()

    # find faces
    if len(face_roi) == 0:
        faces = detector(frame, 1)
    else:
        roi_img = frame[face_roi[0]:face_roi[1], face_roi[2]:face_roi[3]]
        # cv2.imshow('roi', roi_img)
        faces = detector(roi_img)

    # no faces
    if len(faces) == 0:
        print('no faces!')
    
    # find facial landmarks
    for face in faces:
        if len(face_roi) == 0:
            dlib_shape = predictor(frame, face)
            shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])
        else:
            dlib_shape = predictor(roi_img, face)
            shape_2d = np.array([[p.x + face_roi[2], p.y + face_roi[0]]
                                 for p in dlib_shape.parts()])
        for s in shape_2d:
            cv2.circle(frame, center=tuple(s), radius=1, color=(
                255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
        # compute face center
        center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)

        # compute face boundaries
        min_coords = np.min(shape_2d, axis=0)
        max_coords = np.max(shape_2d, axis=0)

        # draw min, max coords
        cv2.circle(frame, center=tuple(min_coords), radius=1,
                   color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
        cv2.circle(frame, center=tuple(max_coords), radius=1,
                   color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
        # compute face size
        face_size = max(max_coords - min_coords)
        face_sizes.append(face_size)
        if len(face_sizes) > 10:
            del face_sizes[0]
        mean_face_size = int(np.mean(face_sizes) * 1.8)
        
        # compute face roi
        face_roi = np.array([int(min_coords[1] - face_size / 2), int(max_coords[1] + face_size / 2),
                             int(min_coords[0] - face_size / 2), int(max_coords[0] + face_size / 2)])
        face_roi = np.clip(face_roi, 0, 10000)
        
        # draw overlay on face
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        if label=="Angry":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Angry_face.png', cv2.IMREAD_UNCHANGED)
            print("Angry")
        elif label=="Disgusting":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Disgusting_face.png', cv2.IMREAD_UNCHANGED)
            print("Disgusting")
        elif label=="Fearful":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Fearful_face.png', cv2.IMREAD_UNCHANGED)
            print("Fearful")
        elif label=="Happy":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Happy_face.png', cv2.IMREAD_UNCHANGED)
            print("Happy")
        elif label=="Sad":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Sad_face.png', cv2.IMREAD_UNCHANGED)
            print("Sad")
        elif label=="Surpring":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/Surpring_face.png', cv2.IMREAD_UNCHANGED)
            print("Surpring")
        elif label=="Neutral":
            overlay = cv2.imread('C:/Users/82109/Desktop/face_detector-master/samples/ryan_transparent.png', cv2.IMREAD_UNCHANGED)
            print("Neutral")
        result = overlay_transparent(
            ori, overlay, center_x + 8, center_y - 25, overlay_size=(mean_face_size, mean_face_size))
        
    # visualize
    cv2.imshow('original', ori)
    #cv2.imshow('facial landmarks', frame)
    cv2.imshow('result', result)

    # Open two windows
    ## Display image ("Emotion Recognition")
    ## Display probabilities of emotion
    cv2.imshow('Emotion Recognition', frame)
    cv2.imshow("Probabilities", canvas)
   
    if cv2.waitKey(1) == ord('q'):
        break
# Clear program and close windows
camera.release()
cv2.destroyAllWindows()

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_36256/3660928469.py:128: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)


Neutral
no faces!
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Sad
Neutral
Neutral
Neutral
Fearful
Fearful
Neutral
Neutral
Neutral
Fearful
Neutral
Fearful
Neutral
Neutral
Fearful
Fearful
Neutral
Neutral
Fearful
Neutral
Neutral
Neutral
Neutral
Neutral
Fearful
Fearful
Neutral
Neutral
Neutral
Neutral
Fearful
Neutral
Fearful
Neutral
Fearful
Neutral
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fearful
Fe

no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!


Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Happy
Happy
Happy
Neutral
Neutral
Neutral
Neutral
Happy
Happy
Happy
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Happy
Neutral
Neutral
Happy
Happy
Happy
Happy
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Ha

no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!


...

no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!


no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!


no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no f

no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!


no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
no faces!
